<a href="https://colab.research.google.com/github/AlisherBlack/LidarDM/blob/main/examples/notebook_kitti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LidarDM's KITTI360 Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzyrianov/LidarDM/blob/main/examples/notebook_kitti.ipynb)

#### Colab setup

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c "nvidia/label/cuda-11.8.0" cuda-toolkit -y

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...
Channels:
 - nvidia/label/cuda-11.8.0
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cuda-toolkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    cuda-cccl-11.8.89          |                0         1.2 MB  nvidia/label/cuda-11.8.0
    cuda-command-line-tools-11.8.0|                0           1

In [ ]:
!git clone https://github.com/vzyrianov/LidarDM.git
%cd LidarDM

# install packages
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install hydra-core
!pip install lightning
!pip install tensorboard
!pip install wandb
!pip install transformers
!pip install matplotlib
!pip install natsort
!pip install pykdtree
!pip install pyntcloud
!pip install pyquaternion
!pip install scikit-learn
!pip install scipy
!pip install tqdm
!pip install open3d
!pip install mediapy
!pip install opencv-python
!pip install diffusers==0.20.2
!pip install nksr==1.0.3+pt20cu117           -f https://nksr.huangjh.tech/whl/torch-2.0.0+cu117.html
!pip install torch-scatter==2.1.2+pt20cu117  -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torchmetrics==1.2.0
!pip install triton==2.0.0

In [ ]:
%cd /content/LidarDM
!pip install -v -e . # for lidardm

# download small asset bank
!wget -O assets.tar.xz "https://uofi.box.com/shared/static/ir42y4a71luia12u17bk5v0fierqrogw.xz"
!tar -xvf assets.tar.xz

# download weights
!mkdir pretrained_models
%cd pretrained_models
!wget -O kitti_weights.zip "https://uofi.box.com/shared/static/tc4hppt38ryy5rsgthiw4q50dxtu1w2f.zip"
!unzip kitti_weights.zip

%cd /content/LidarDM

#### Env setup

In [ ]:
import os

import mediapy
from examples.utils import get_visualization_videos

from lidardm import PROJECT_DIR
from lidardm.core.services.sampling import sample_uncond_kitti, instantiate_unconditional_model_kitti
from lidardm.lidar_generation.scene_composition.compositors import GenerativeCompositor

#### Generating the static world

In [ ]:
model = instantiate_unconditional_model_kitti(os.path.join(PROJECT_DIR, "pretrained_models/kitti360/scene_gen/kfsvae_kl1e-7.ckpt"),
                                              os.path.join(PROJECT_DIR, "pretrained_models/kitti360/scene_gen/kfsunet_b.ckpt"))
model = model.cuda()
mesh = sample_uncond_kitti(model)

#### Composing underlying 4D scene

If you are running this notebook via SSH, it is likely that Open3D headless rendering won't work. set `USING_SSH = True` for (not-as-pretty-but-working) visualization

In [ ]:
USING_SSH = False

compositor = GenerativeCompositor(mesh=mesh, n_frames=20)
videos = get_visualization_videos(compositor, USING_SSH)
mediapy.show_videos(videos, fps=10, height=400)